In [ ]:
#%pip install -r requirements.txt

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=400.0)

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("data").load_data()


# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')



c:\automatization\conf7nov\ollama\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(documents)

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [ ]:
# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
from IPython.display import Markdown, display


# save to disk

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



In [ ]:

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


In [ ]:
print(chroma_collection.count())

In [ ]:

# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store
)


In [ ]:
chroma_collection.count()

In [ ]:

# Query Data from the persisted index
query_engine = index.as_query_engine()


In [ ]:
prompt = 'What is the bayraktar?'

response = query_engine.query(prompt)
display(Markdown(f"<b>{response}</b>"))